Dependencies:

numpy: 1.21.0\
cvxpy: 1.1.14\
json : 2.0.9

* Demands: 
    * Day 1: 75
    * Day 2: 150
    * Day 3: 100
    * Day 4: 60
* Unit production cost:
    * Day 1: \$4
    * Day 2: \$5
    * Day 3: \$7
    * Day 4: \$4
* Inventory cost: $1 per unit per day

How many units should we produce per day to minimize the total cost?

# Problem statement

**Input parameters**

$d_t$: demand in day $t$, $t=1,..., 4$

$c_t$: unit cost in day $t$, $t=1,..., 4$

In [136]:
d = [75, 150, 100, 60]
c = [4, 5, 7, 4]

**Decision variables**:

$q_t$: production quantity of day $t$, $t=1, ..., 4$

$e_t$: ending inventory of day $t, t=0,1, ..., 4$

In [137]:
import cvxpy as cp

In [138]:
t = 4
q = cp.Variable(t, integer=True)
e = cp.Variable(t+1, integer=True)

**Objectives:**

\begin{align}
\min \sum_{t=1}^4 c_t q_t + e_t \times 1
\end{align}

$$e_t \times 1 = e_t$$

$$
\sum_{t=1}^4 c_t q_t = 4\times75 + 8 \times 150 + 12 \times 100 + 7 \times 60 \\
= \begin{bmatrix}
4& 8& 12& 7
\end{bmatrix} \begin{bmatrix}
75\\ 
150\\ 
100\\
60
\end{bmatrix} \\
$$

In [139]:
obj = cp.Minimize(c @ q + sum(e))

**Constraints:**

The ending inventory is equal to the production quantity of day $t$ subtract the demand in day $t$:

$$
e_1 = e_0 + q_1 - d_1\\
e_2 = e_1 + q_2 - d_2\\
$$
In general, we have:
$$
e_t = e_{t-1} + q_t - d_t \;\; \forall t = 1, 2, 3, 4\\
$$

The number of units in inventory at the beginning of day 1 is 0:
$$e_0 = 0$$

The number of units in inventory cannot be negative:
$$e_t \geq 0\;\; \forall t = 0, 1, 2, 3, 4$$

Production quantitiy cannot be negative:
$$q_t \geq 0\;\; \forall t = 1, 2, 3, 4$$

In [155]:
constraints = [e[0] == 0, e[1:] == e[:-1] + q - d, e >= 0, q >= 0]

<IPython.core.display.Javascript object>

# Solve

In [141]:
prob = cp.Problem(obj, constraints)

In [142]:
prob.solve()

1890.0

In [143]:
prob.status

'optimal'

In [144]:
print(q.value)

[ 75. 250.   0.  60.]


In [145]:
print(e.value)

[  0.   0. 100.   0.   0.]


# Experiment with Other Possible Combinations

In [146]:
import numpy as np

In [147]:
def calculate_inventory(demands: np.array, quantities: np.array):
    """Calculate the ending inventory based on the demand and the production quantities for each day"""

    inventory = []
    new_inventory = 0
    for i in range(len(quantities)):
        new_inventory = new_inventory + quantities[i] - demands[i]
        inventory.append(new_inventory)
    return inventory

In [156]:
def get_cost(demands: np.array, cost: np.array, quantities: np.array):
    """Get the total cost based on the production quantities"""
    inventory_per_day = calculate_inventory(demands, quantities)
    return sum(cost * quantities + inventory_per_day * 1)

<IPython.core.display.Javascript object>

Produce everything in the first day

In [157]:
demands = np.array([75, 150, 100, 60])
cost = np.array([4, 5, 7, 4])
quantities = np.array([385, 0, 0, 0])
get_cost(demands, cost, quantities)

2070

<IPython.core.display.Javascript object>

Produce everything in the first day and second day

In [158]:
quantities = np.array([300, 85, 0, 0])
get_cost(demands, cost, quantities)

2070

<IPython.core.display.Javascript object>

Produce just enough to fulfill the demand for each day

In [159]:
quantities = np.array([75, 150, 100, 60])
get_cost(demands, cost, quantities)

1990

<IPython.core.display.Javascript object>

Produce everything in the first day and the last day

In [152]:
quantities = np.array([325, 0, 0, 60])
get_cost(demands, cost, quantities)

1890